### IR Book 16.3 Concept Code

This code implements the Bernoulli EM algorithm as described in the IR Book, example 16.3. This exercise is useful to ensure an understanding of the algorithm and to check functionality. There are a few bits that aren't explained will in the book but become apparent when working with this code. 

- During the computation of the $r_{n,k}$ classifications it is best to use log(probabilities) in order to prevent underflow

- The use of $\epsilon$ as a smoothing parameter is crucial to the good behavior of the algorithm and to avoid either a divide by zero or a log(0) problem. In the IR book $\epsilon$ is set to 0.0001. Setting to smaller values causes the algorithm to take more iterations to converge to the same solution as in the book.

In a Bernoulli Mixture Model a document is a vector of Booleans indicating the presence of a term.

The conditional probability of a document given a set of parameters is given by:

$$P(d \;|\;\theta) = \sum_{k=1}^{K}\alpha_k(\prod_{tm \in d} q_{mk})(\prod_{tm \notin d}(1 - q_{mk}))$$

This is the sum for each class of the product of the probabilities of the terms in a document with 1 minus the probabilities of the terms not in the document.

The probability that a document from cluster $\omega_{k}$ containts term $t_{m}$ is given by:

$$q_{mk} = P(U_{m} = 1 \; | \;\omega_{k})$$

The prior $\alpha_{k}$ of cluster $\omega_{k}$ is the probability document $d$ is in $\omega_{k}$ if we have no other information about it.

When we don't know the classifications of the documents we can use Expectation Maximization iteratively to arrive at classifications, $r_{nk}$.

**E Step**

$$\tag{1} r_{nk} = \frac{\alpha_{k}\left(\prod_{tm \in d_n}q_{mk})(\prod_{tm \notin d_n}(1-q_{mk})\right)}{\sum_{k=1}^{K}\alpha_{k}\left(\prod_{tm \in d_n}q_{mk})(\prod_{tm \notin d_n}(1-q_{mk})\right)}$$

The actual computation as implemented by taking the sum of the log probabilities as opposed to the products of the probabilities themselves. This is necessary because once you are dealing with many terms, multiplying a lot of small numbers results in numeric underflow. Also, in order to prevent taking the log(0), which will happen in the first iteration, a very small number, $\delta$, is added to the probability before taking the log. So we end up with:

$$\tag{1a} \frac{\alpha_{k}exp{\left(\sum_{tm \in d_n}log(q_{mk}) + \sum_{tm \notin d_n}log(1-q_{mk}) \right)}}{\sum_{k}\alpha_{k} exp{\left(\sum_{tm \in d_n} log(q_{mk}) + \sum_{tm \notin d_n}log(1-q_{mk}) \right)}}$$





$$\tag{1a} exp\left[log(\alpha_{k}) +\sum_{tm \in d_n}log(q_{mk}) + \sum_{tm \notin d_n}log(1-q_{mk}) -log\sum_{k} exp\left( log(\alpha_{k}) + \sum_{tm \in d_n} log(q_{mk}) + \sum_{tm \notin d_n}log(1-q_{mk})\right)\right]$$







$$\tag{1a} \frac{\alpha_{k}exp{\left(\sum_{tm \in d_n}log(q_{mk}+\delta) + \sum_{tm \notin d_n}log(1-q_{mk}+\delta) \right)}}{\sum_{k}\alpha_{k} exp{\left(\sum_{tm \in d_n} \left(log(q_{mk}+\delta) + \sum_{tm \notin d_n}log(1-q_{mk}+\delta) \right) \right)}}$$

In the code below, a single pass is made to calculate the $r$ class soft assignments so that the terms are calculated only once so it may not be obvious what's going on.

_Note: the $\delta$ values are important for the algorithm to converge. If you take a straight calculation of the $r_{1,1}$ term in the first iteration you will end up with a divide by zero using the original equations. By including the $\delta$ term this is avoided and you'll see that the result is very close to 1_

**M Step**

$$\tag{2} q_{mk} = \frac{\sum_{n=1}^{N}(r_{nk} + \epsilon)I(t_m \in d_n)}{\sum_{i=1}^{N}(r_{nk} + \epsilon)}$$

$I(t_{m} \in d_{n}) = 1$ if term is an element of document n and 0 otherwise.

$\epsilon$ is for smoothing

Finally, priors are updated per iteration as:

$$\tag{3} \alpha_k = \frac{\sum_{n=1}^{N}r_{nk}}{N}$$




In [1]:
# save this to a file - it will be handy for MR testing later
with open('test.txt','w') as outfile:
    outfile.write('hot chocolate cocoa beans\n')
    outfile.write('cocoa ghana africa\n')
    outfile.write('beans harvest ghana\n')
    outfile.write('cocoa butter\n')
    outfile.write('butter truffles\n')
    outfile.write('sweet chocolate\n')
    outfile.write('sweet sugar\n')
    outfile.write('sugar cane brazil\n')
    outfile.write('sweet sugar beet\n')
    outfile.write('sweet cake icing\n')
    outfile.write('cake black forest\n')

In [1]:
import re
from math import log
import numpy as np

# read the documents. Each document consists of a list of words.
documents = []
with open('test.txt', 'r') as docfile:
    for line in docfile.readlines():
        documents.append(re.split(' ', line.strip()))

classes = 2
r = [[None] * len(documents), [None] * len(documents)]

# set our initial conditions (r_6,1 = 1.0 and r_7,1 = 0.0 and the converse for the other class)
r[0][5] = r[1][6] = 1.0
r[0][6] = r[1][5] = 0.0

# initialize the priors
alpha = [0.0] * classes

# delta is to keep the arithmetic well behaved - it is not the smoothing factor
delta = 0

# epsilon is for smoothing in Equation 2 and 3 (16.16 in the IR Book)
epsilon = 0.0001

# conditional term probabilities
qm = {}

# compute alphas - Equation 3
def compute_alphas(alphas):
    for k in range(classes):
        alphas[k] = sum([x+epsilon for x in r[k] if x is not None])/len([x for x in r[k] if x is not None])


# compute inverted postings list
# this is handy for the computation of the qm's
def compute_postings():
    postings = {}
    for i in range(len(documents)):
        if r[0][i] is not None:
            for word in documents[i]:
                if word not in postings:
                    postings[word] = [i]
                else:
                    if i not in postings[word]:
                        postings[word].append(i)
    return postings

# compute qm's - Equation 2
def compute_next_qms(qm):
    for k in range(classes):
        for i in range(len(r[k])):
            if r[k][i] is not None:
                for word in documents[i]:
                    # use smoothing value epsilon
                    if word not in qm:
                        qm[word] = {k: sum([r[k][j]+epsilon for j in postings[word]]) / \
                                    sum([x+epsilon for x in r[k] if x is not None])}
                    else:
                        qm[word][k] = sum([r[k][j]+epsilon for j in postings[word]]) / \
                                    sum([x+epsilon for x in r[k] if x is not None])

# compute qm's - Equation 2 version 2
def compute_qms(qm):
    # for each of the classes
    for k in range(classes):
        # for each of the terms
        for term in postings:
            numerator = 0.0
            # for each of the documents in the postings list for the term
            for doc in postings[term]:
                numerator += r[k][doc]
        qm[k][term]=numerator/sum(filter(None,r[k]))
        
            

# compute next iteration of r's - Equation 1a
# note - need to do log(probability)
def compute_next_r(rs):
    for i in range(len(documents)):
        vocab_words_in_doc = []
        p = np.zeros((classes, 2))
        
        # find all vocab words in the doc. Note: there may be none.
        for word in documents[i]:
            if word in qm:
                vocab_words_in_doc.append(word)
                for k in range(classes):
                    # prevent math errors by not taking log(0)
                    p[k][0] += np.log(qm[word][k] + delta)
        if len(vocab_words_in_doc) > 0:
            # find all vocab words not in doc for all classes at the same time
            for word in qm:
                if word not in vocab_words_in_doc:
                    for k in range(classes):
                        # prevent math errors by not taking log(0)
                        p[k][1] += np.log(1-qm[word][k] + delta)                        
            # compute the denominator of Equation 1a for all classes
            denom = 0.0
            for k in range(classes):
                denom += alpha[k]*np.exp(p[k][0]+p[k][1])
                
            # compute the new r of Equation 1a for all classes
            for k in range(classes):
                rs[k][i] = alpha[k]*np.exp(p[k][0]+p[k][1])/denom

        else:
            # set to prior in case of no information
            for k in range(classes):
                rs[k][i] = alpha[k]

# iterate
for _ in range(25):
    compute_alphas(alpha)
    postings = compute_postings()
    compute_next_qms(qm)
    compute_next_r(r)

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:83: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:91: RuntimeWarning: invalid value encountered in double_scalars


In [2]:
# if you take out rounding you can see that these values are not exact
print 'Soft classifications\n',np.around(r, 2)
print '\nPriors (alpha)',np.around(alpha,2)


Soft classifications
[[ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan]]

Priors (alpha) [ nan  nan]


In [208]:
import re
from math import log
import numpy as np

# read the documents. Each document consists of a list of words. From reading each document,
# construct a list of unique terms and reconstruct the document as a list of indexes into
# the unique terms. This is an alternative to creating a boolean vector where the index of the
# boolean corresponds to the index of the term. If there are duplicated terms in a document
# we only include one instance as we only care about whether the term is in the doc
#
# documents is a dictionary so that it uses a document id as a key
documents = {}
terms = []
with open('test.txt', 'r') as docfile:
    i = 1
    for line in docfile.readlines():
        doc = []
        doc_words = re.split(' ', line.strip())
        for word in doc_words:
            if word not in terms:
                terms.append(word)
            if terms.index(word) not in doc:
                doc.append(terms.index(word))
        documents[i] = doc
        i += 1

In [265]:
classes = 2

r = {}
for k in range(classes):
    for doc in documents:
        if doc not in r:
            r[doc] = {k : None}
        else:
            r[doc][k] = None


# set our initial conditions (r_6,1 = 1.0 and r_7,1 = 0.0 and the converse for the other class)
r[6][0] = r[7][1] = 1.0
r[6][1] = r[7][0] = 0.0

# initialize the priors
alpha = [0.0] * classes

delta = 1E-12

# epsilon is for smoothing in Equation 2 and 3 (16.16 in the IR Book)
epsilon = 0.0001

# conditional term probabilities
qm = np.zeros((classes, len(terms)))

# the vocabulary under consideration changes with each iteration
vocab = []

In [210]:
# compute alphas - Equation 3
def compute_alphas():
    for k in range(classes):
        alpha[k] = 0.0
        n = 0
        for doc in r:
            if r[doc][k] is not None:
                alpha[k] += r[doc][k]
                n = n+1
        alpha[k] = alpha[k]/n

In [211]:
# compute inverted postings list
# this is handy for the computation of the qm's
def compute_postings():
    postings = {}
    for doc in documents:
        for term_idx in documents[doc]:
            if term_idx not in postings:
                postings[term_idx] = [doc]
            else:
                if doc not in postings[term_idx]:
                    postings[term_idx].append(doc)
    return postings

In [212]:
documents

{1: [0, 1, 2, 3],
 2: [2, 4, 5],
 3: [3, 6, 4],
 4: [2, 7],
 5: [7, 8],
 6: [9, 1],
 7: [9, 10],
 8: [10, 11, 12],
 9: [9, 10, 13],
 10: [9, 14, 15],
 11: [14, 16, 17]}

In [213]:
terms

['hot',
 'chocolate',
 'cocoa',
 'beans',
 'ghana',
 'africa',
 'harvest',
 'butter',
 'truffles',
 'sweet',
 'sugar',
 'cane',
 'brazil',
 'beet',
 'cake',
 'icing',
 'black',
 'forest']

In [249]:
r

{1: {0: None, 1: None},
 2: {0: None, 1: None},
 3: {0: None, 1: None},
 4: {0: None, 1: None},
 5: {0: None, 1: None},
 6: {0: 1.0, 1: 0.0},
 7: {0: 0.0, 1: 1.0},
 8: {0: None, 1: None},
 9: {0: None, 1: None},
 10: {0: None, 1: None},
 11: {0: None, 1: None}}

In [215]:
postings = compute_postings()

In [225]:
postings

{0: [1],
 1: [1, 6],
 2: [1, 2, 4],
 3: [1, 3],
 4: [2, 3],
 5: [2],
 6: [3],
 7: [4, 5],
 8: [5],
 9: [6, 7, 9, 10],
 10: [7, 8, 9],
 11: [8],
 12: [8],
 13: [9],
 14: [10, 11],
 15: [10],
 16: [11],
 17: [11]}

In [217]:
# compute qm's - Equation 2 version 2
def compute_qms():
    # for each of the classes
    for k in range(classes):
        # compute the denominator
        denominator = 0.0
        for doc in r:
            if r[doc][k] is not None:
                denominator += r[doc][k]+epsilon
        # compute the numerator across all terms
        for term in postings:
            numerator = 0.0
            # for each of the documents in the postings list for the term
            for doc in postings[term]:
                if r[doc][k] is not None:
                    numerator += r[doc][k] + epsilon
#                print k,term,doc, numerator
            qm[k][term]=numerator/denominator
        

In [226]:
compute_qms()
print 'africa 1 ',np.round(qm[0][terms.index('africa')],2)
print 'africa 2 ',np.round(qm[1][terms.index('africa')],2)
print 'brazil 1 ',np.round(qm[0][terms.index('brazil')],2)
print 'brazil 2 ',np.round(qm[1][terms.index('brazil')],2)
print 'cocoa 1 ',np.round(qm[0][terms.index('cocoa')],2)
print 'cocoa 2 ',np.round(qm[1][terms.index('cocoa')],2)
print 'sugar 1 ',np.round(qm[0][terms.index('sugar')],2)
print 'sugar 2 ',np.round(qm[1][terms.index('sugar')],2)
print 'sweet 1 ',np.round(qm[0][terms.index('sweet')],2)
print 'sweet 2 ',np.round(qm[1][terms.index('sweet')],2)

africa 1  0.0
africa 2  0.0
brazil 1  0.0
brazil 2  0.0
cocoa 1  0.0
cocoa 2  0.0
sugar 1  0.0
sugar 2  1.0
sweet 1  1.0
sweet 2  1.0


In [238]:
np.around(qm,2)

array([[ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.]])

In [227]:
compute_alphas()
np.around(alpha,2)

array([ 0.5,  0.5])

In [260]:
def update_vocabulary():
    # find the documents with a classification score and only use the terms in 
    # those documents to compute the next r values
    for doc in r:
        # doesn't matter which class it is
        if r[doc][0] is not None:
            for term in documents[doc]:
                if term not in vocab:
                    vocab.append(term)

In [264]:
def compute_r():    
    # compute the denominator once since it is summed across all classes
    denominator = 0.0
    for k in range(classes):
        dt = 0.0
        for term in vocab:
            dt += np.log(qm[k][term]+delta) + np.log(1-qm[k][term]+delta)
            print '-----',term, qm[k][term]
        denominator = denominator + alpha[k]*np.exp(dt)

    print denominator
    # compute the new document soft class assignment, r, for each class, k
    for k in range(classes):        
        for doc in documents:
            # if this document has an terms in the vocabulary, compute the new r
            if not set(documents[doc]).isdisjoint(vocab):
                print doc
                numerator = 0.0
                for term in vocab:
                    if term in documents[doc]:
                        numerator += np.log(qm[k][term]+delta)
                    else:
                        numerator += np.log(1.0 - qm[k][term]+delta)
                r[doc][k] = (alpha[k]*np.exp(numerator))/denominator
                print doc,' ',k,' ',(alpha[k]*np.exp(numerator))/denominator
            
            
        
            

In [266]:
update_vocabulary()
vocab

[9, 1, 10]

In [267]:
compute_r()
r

----- 9 0.0
----- 1 0.0
----- 10 0.0
----- 9 0.0
----- 1 0.0
----- 10 0.0
0.0
1
1   0   nan
6
6   0   nan
7
7   0   nan
8
8   0   nan
9
9   0   nan
10
10   0   nan
1
1   1   nan
6
6   1   nan
7
7   1   nan
8
8   1   nan
9
9   1   nan
10
10   1   nan


/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:24: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:25: RuntimeWarning: invalid value encountered in double_scalars


{1: {0: nan, 1: nan},
 2: {0: None, 1: None},
 3: {0: None, 1: None},
 4: {0: None, 1: None},
 5: {0: None, 1: None},
 6: {0: nan, 1: nan},
 7: {0: nan, 1: nan},
 8: {0: nan, 1: nan},
 9: {0: nan, 1: nan},
 10: {0: nan, 1: nan},
 11: {0: None, 1: None}}

In [1]:
# compute next iteration of r's - Equation 1a
# note - need to do log(probability)
def compute_next_r(rs):
    for i in range(len(documents)):
        vocab_words_in_doc = []
        p = np.zeros((classes, 2))
        
        # find all vocab words in the doc. Note: there may be none.
        for word in documents[i]:
            if word in qm:
                vocab_words_in_doc.append(word)
                for k in range(classes):
                    # prevent math errors by not taking log(0)
                    p[k][0] += np.log(qm[word][k] + delta)
        if len(vocab_words_in_doc) > 0:
            # find all vocab words not in doc for all classes at the same time
            for word in qm:
                if word not in vocab_words_in_doc:
                    for k in range(classes):
                        # prevent math errors by not taking log(0)
                        p[k][1] += np.log(1-qm[word][k] + delta)                        
            # compute the denominator of Equation 1a for all classes
            denom = 0.0
            for k in range(classes):
                denom += alpha[k]*np.exp(p[k][0]+p[k][1])
                
            # compute the new r of Equation 1a for all classes
            for k in range(classes):
                rs[k][i] = alpha[k]*np.exp(p[k][0]+p[k][1])/denom

        else:
            # set to prior in case of no information
            for k in range(classes):
                rs[k][i] = alpha[k]

# iterate
for _ in range(25):
    compute_alphas(alpha)
    postings = compute_postings()
    compute_next_qms(qm)
    compute_next_r(r)

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:83: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:91: RuntimeWarning: invalid value encountered in double_scalars
